In [20]:
from aocd.models import Puzzle
import networkx as nx
puzzle = Puzzle(year=2023, day=17)
data = puzzle.input_data
%run helper.ipynb

In [21]:
grid = Grid(data)

dir_map = {(0,1): 0, (1,0): 1, (0,-1): 2, (-1, 0): 3}
start_node = (0,0)
end_node = (len(grid)-1,len(grid[0])-1)

def point_diff(a,b):
    return (a[0] - b[0], a[1] - b[1])

def current_dir(state):
    for k, v in dir_map.items():
        if state[v] != 0:
            return k

def is_opposite(a, b):
    return (a[0] == -b[0] and a[1] == -b[1])


def make_graph(grid, min_travel, max_travel):
    graph = nx.DiGraph()
    poss_states = flatten([[(i,0,0,0), (0,i,0,0), (0,0,i,0), (0,0,0,i)] for i in range(1, max_travel + 1)])

    for row in range(len(grid)):
        for col in range(len(grid[0])):
            for state in poss_states:
                graph.add_node((row, col, state))

    for row in range(len(grid)):
        for col in range(len(grid[0])):
            for state in poss_states:
                sides = grid.sides((row, col))
                values = grid.sides((row, col), vals=True)
                for i in range(len(sides)):
                    curr_node = (row, col, state)
                    diff = point_diff(sides[i], (row,col))
                    prev_dir_value = dir_map[diff]
                    new_state = [0, 0, 0, 0]
                    prev_speed = max(state)
                    if min_travel > 0 and prev_speed < min_travel and state[prev_dir_value] == 0:
                        continue
                    if state[prev_dir_value] == 0:
                        new_state[prev_dir_value] = 1
                    elif state[prev_dir_value] < max_travel:
                        new_state[prev_dir_value] = state[prev_dir_value] + 1
                    else:
                        continue
                    if is_opposite(current_dir(state), current_dir(new_state)):
                        continue
                    if sides[i] == end_node and (min_travel == 0 or max(new_state) > min_travel):
                        next_node = end_node
                    else:
                        next_node = (sides[i][0], sides[i][1], tuple(new_state))
                    graph.add_edge(curr_node, next_node, weight=values[i])

    graph.add_node(start_node)
    graph.add_edge(start_node, (0,1, (1,0,0,0)), weight=grid[(0,1)])
    graph.add_edge(start_node, (1,0, (0,1,0,0)), weight=grid[(1,0)])
    return graph

In [ ]:
graph1 = make_graph(grid, 0, 3, part_1_states)
p = nx.shortest_path(graph1, source=start_node, target=end_node, weight="weight")
puzzle.answer_a = nx.path_weight(graph1, p, weight="weight")

In [ ]:
graph2 = make_graph(grid, 4, 10, part_2_states)
p = nx.shortest_path(graph2, source=start_node, target=end_node, weight="weight")
puzzle.answer_a = nx.path_weight(graph2, p, weight="weight")